In [1]:
import pandas
import numpy
from sklearn import mixture
from sklearn.mixture import GaussianMixture
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
fllst=['20170317KW KW Con.csv']

In [7]:
# gmm 4D4P GMM - representing data as heatmap


def gateDNA_Event_length(fle,cms):
    df=pandas.read_csv(fle,delimiter=',')
    filtereddf = df[(numpy.log(df["Event_length"]+1)<4.5) & ((numpy.log(df["Ir191Di"]+1)>7.5) | (numpy.log(df["Ir193Di"]+1)>8.0))]
    data = {}
    for k in cms.keys():
        data[cms[k]] = filtereddf[k]
    dtafrm=pandas.DataFrame(data)
    return dtafrm

def predictPopulations(dt,ngroups):
    Classifier = dt.sample(frac=0.25)    
    model=GaussianMixture(n_components=ngroups,tol=0.001,n_init=5).fit(numpy.log(Classifier+1))
    predictions = model.predict(numpy.log(dt+1))
    counts = []
    for p in range(ngroups):
        counts.append(sum(predictions==p))
    means = pandas.DataFrame(model.means_)
    means.columns = Classifier.columns # assign column names
    return (means,counts)

typmrks = {
    "Y89Di":"CD3",
    "Bi209Di":"CD4",
    "Nd142Di":"CD19",
   "Er168Di":"CD8",
}

dt = gateDNA_Event_length("20170317KW KW Con.csv",typmrks)
ngroups = 4 # define the number of populations
A=predictPopulations(dt,ngroups)
B=stats.zscore(A[0]) #get z-score of means per column for heatmap 
df=A[0] #define a dataframe to get the column names for the heatmap ("xticklabels=df.columns")

In [ ]:
aap=sns.heatmap(B,cmap='Blues',linecolor='black', linewidths='0.1', yticklabels=A[1], xticklabels=df.columns) # on z-score